In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from collections import Counter
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd
import re
from torch.optim.lr_scheduler import ReduceLROnPlateau

class TextPreprocessor:
    def __init__(self, max_vocab_size=15000, max_seq_length=128):
        self.max_vocab_size = max_vocab_size
        self.max_seq_length = max_seq_length
        self.word2idx = {'<PAD>': 0, '<UNK>': 1}
        self.idx2word = {0: '<PAD>', 1: '<UNK>'}
        self.word_counts = Counter()
        
    def clean_text(self, text):
        text = str(text).lower()
        text = re.sub(r'[^a-zA-Z0-9\s.,!?]', '', text)
        text = ' '.join(text.split())
        return text
        
    def fit(self, texts):
        for text in texts:
            cleaned_text = self.clean_text(text)
            words = cleaned_text.split()
            self.word_counts.update(words)
        
        vocab_words = [word for word, count in self.word_counts.most_common(self.max_vocab_size - 2)]
        for word in vocab_words:
            idx = len(self.word2idx)
            self.word2idx[word] = idx
            self.idx2word[idx] = word
    
    def transform(self, texts):
        sequences = []
        for text in texts:
            cleaned_text = self.clean_text(text)
            words = cleaned_text.split()
            seq = [self.word2idx.get(word, self.word2idx['<UNK>']) for word in words[:self.max_seq_length]]
            seq = seq + [self.word2idx['<PAD>']] * (self.max_seq_length - len(seq))
            sequences.append(seq)
        return torch.tensor(sequences)

class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

class ImprovedTextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_classes, num_layers=2):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.embedding_dropout = nn.Dropout(0.2)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers=num_layers, batch_first=True, bidirectional=True, dropout=0.2 if num_layers > 1 else 0)
        self.attention = nn.Linear(hidden_dim * 2, 1)
        self.fc1 = nn.Linear(hidden_dim * 2, hidden_dim)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.dropout1 = nn.Dropout(0.3)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim // 2)
        self.bn2 = nn.BatchNorm1d(hidden_dim // 2)
        self.dropout2 = nn.Dropout(0.2)
        self.fc3 = nn.Linear(hidden_dim // 2, num_classes)
        
    def attention_net(self, lstm_output):
        attention_weights = torch.softmax(self.attention(lstm_output), dim=1)
        context_vector = torch.sum(attention_weights * lstm_output, dim=1)
        return context_vector
        
    def forward(self, x):
        embedded = self.embedding(x)
        embedded = self.embedding_dropout(embedded)
        lstm_out, _ = self.lstm(embedded)
        attn_out = self.attention_net(lstm_out)
        x = self.fc1(attn_out)
        x = self.bn1(x)
        x = torch.relu(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = torch.relu(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs, device):
    best_f1 = 0
    patience = 5
    patience_counter = 0
    
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        train_preds = []
        train_labels = []
        
        for batch_texts, batch_labels in train_loader:
            batch_texts, batch_labels = batch_texts.to(device), batch_labels.to(device)
            optimizer.zero_grad()
            outputs = model(batch_texts)
            loss = criterion(outputs, batch_labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            train_loss += loss.item()
            predictions = torch.argmax(outputs, dim=1)
            train_preds.extend(predictions.cpu().numpy())
            train_labels.extend(batch_labels.cpu().numpy())
        
        model.eval()
        val_loss = 0
        val_preds = []
        val_labels = []
        
        with torch.no_grad():
            for batch_texts, batch_labels in val_loader:
                batch_texts, batch_labels = batch_texts.to(device), batch_labels.to(device)
                outputs = model(batch_texts)
                loss = criterion(outputs, batch_labels)
                val_loss += loss.item()
                predictions = torch.argmax(outputs, dim=1)
                val_preds.extend(predictions.cpu().numpy())
                val_labels.extend(batch_labels.cpu().numpy())
        
        train_precision, train_recall, train_f1, _ = precision_recall_fscore_support(train_labels, train_preds, average='binary')
        train_accuracy = accuracy_score(train_labels, train_preds)
        
        val_precision, val_recall, val_f1, _ = precision_recall_fscore_support(val_labels, val_preds, average='binary')
        val_accuracy = accuracy_score(val_labels, val_preds)
        
        scheduler.step(val_loss)
        
        print(f'Epoch {epoch+1}/{num_epochs}:')
        print(f'Train Loss: {train_loss/len(train_loader):.4f}, Accuracy: {train_accuracy:.4f}, F1: {train_f1:.4f}')
        print(f'Val Loss: {val_loss/len(val_loader):.4f}, Accuracy: {val_accuracy:.4f}, F1: {val_f1:.4f}')
        print(f'Val Precision: {val_precision:.4f}, Recall: {val_recall:.4f}\n')
        
        if val_f1 > best_f1:
            best_f1 = val_f1
            torch.save({
                'model_state_dict': model.state_dict(),
                'preprocessor': preprocessor,
                'config': config
            }, 'best_model.pt')
            patience_counter = 0
        else:
            patience_counter += 1
            
        if patience_counter >= patience:
            print(f'Early stopping triggered after epoch {epoch+1}')
            break

config = {
    'max_vocab_size': 15000,
    'max_seq_length': 128,
    'embed_dim': 256,
    'hidden_dim': 256,
    'batch_size': 16,
    'learning_rate': 0.001,
    'num_epochs': 20,
    'num_lstm_layers': 2
}

train_df = pd.read_csv('train.tsv', sep='\t')
test_df = pd.read_csv('test.tsv', sep='\t')

label_mapping = {'NOCUOUS': 0, 'INNOCUOUS': 1}
train_df['Detected as'] = train_df['Detected as'].map(label_mapping)
test_df['Detected as'] = test_df['Detected as'].map(label_mapping)

preprocessor = TextPreprocessor(max_vocab_size=config['max_vocab_size'], max_seq_length=config['max_seq_length'])
preprocessor.fit(train_df['Sentence'])

X_train = preprocessor.transform(train_df['Sentence'])
X_val = preprocessor.transform(test_df['Sentence'])
y_train = torch.tensor(train_df['Detected as'].values)
y_val = torch.tensor(test_df['Detected as'].values)

train_dataset = TextDataset(X_train, y_train)
val_dataset = TextDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config['batch_size'])

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ImprovedTextClassifier(
    vocab_size=len(preprocessor.word2idx),
    embed_dim=config['embed_dim'],
    hidden_dim=config['hidden_dim'],
    num_classes=2,
    num_layers=config['num_lstm_layers']
).to(device)

class_counts = torch.bincount(y_train)
class_weights = 1.0 / class_counts
class_weights = class_weights / class_weights.sum()
class_weights = class_weights.to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.AdamW(model.parameters(), lr=config['learning_rate'], weight_decay=0.01)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)

# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler,
           config['num_epochs'], device)


Epoch 1/20:
Train Loss: 0.7980, Accuracy: 0.5180, F1: 0.4553
Val Loss: 0.6972, Accuracy: 0.4384, F1: 0.6095
Val Precision: 0.4384, Recall: 1.0000

Epoch 2/20:
Train Loss: 0.5651, Accuracy: 0.7626, F1: 0.7556
Val Loss: 0.7046, Accuracy: 0.4384, F1: 0.6095
Val Precision: 0.4384, Recall: 1.0000

Epoch 3/20:
Train Loss: 0.3815, Accuracy: 0.8561, F1: 0.8551
Val Loss: 0.7101, Accuracy: 0.4384, F1: 0.6095
Val Precision: 0.4384, Recall: 1.0000

Epoch 00004: reducing learning rate of group 0 to 5.0000e-04.
Epoch 4/20:
Train Loss: 0.1724, Accuracy: 0.9496, F1: 0.9466
Val Loss: 0.7195, Accuracy: 0.4521, F1: 0.6078
Val Precision: 0.4429, Recall: 0.9688

Epoch 5/20:
Train Loss: 0.1531, Accuracy: 0.9496, F1: 0.9466
Val Loss: 0.6510, Accuracy: 0.6849, F1: 0.5818
Val Precision: 0.6957, Recall: 0.5000

Epoch 6/20:
Train Loss: 0.0977, Accuracy: 0.9712, F1: 0.9697
Val Loss: 0.6682, Accuracy: 0.6575, F1: 0.5283
Val Precision: 0.6667, Recall: 0.4375

Epoch 7/20:
Train Loss: 0.0683, Accuracy: 0.9856, F1: 0.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from collections import Counter
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd
import re
from torch.optim.lr_scheduler import ReduceLROnPlateau

class TextPreprocessor:
    def __init__(self, max_vocab_size=15000, max_seq_length=128):
        self.max_vocab_size = max_vocab_size
        self.max_seq_length = max_seq_length
        self.word2idx = {'<PAD>': 0, '<UNK>': 1}
        self.idx2word = {0: '<PAD>', 1: '<UNK>'}
        self.word_counts = Counter()
        
    def clean_text(self, text):
        text = str(text).lower()
        text = re.sub(r'[^a-zA-Z0-9\s.,!?]', '', text)
        text = ' '.join(text.split())
        return text
        
    def fit(self, texts):
        for text in texts:
            cleaned_text = self.clean_text(text)
            words = cleaned_text.split()
            self.word_counts.update(words)
        
        vocab_words = [word for word, count in self.word_counts.most_common(self.max_vocab_size - 2)]
        for word in vocab_words:
            idx = len(self.word2idx)
            self.word2idx[word] = idx
            self.idx2word[idx] = word
    
    def transform(self, texts):
        sequences = []
        for text in texts:
            cleaned_text = self.clean_text(text)
            words = cleaned_text.split()
            seq = [self.word2idx.get(word, self.word2idx['<UNK>']) for word in words[:self.max_seq_length]]
            seq = seq + [self.word2idx['<PAD>']] * (self.max_seq_length - len(seq))
            sequences.append(seq)
        return torch.tensor(sequences)

class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

class ImprovedTextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_classes, num_layers=2):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.embedding_dropout = nn.Dropout(0.2)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers=num_layers, batch_first=True, bidirectional=True, dropout=0.2 if num_layers > 1 else 0)
        self.attention = nn.Linear(hidden_dim * 2, 1)
        self.fc1 = nn.Linear(hidden_dim * 2, hidden_dim)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.dropout1 = nn.Dropout(0.3)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim // 2)
        self.bn2 = nn.BatchNorm1d(hidden_dim // 2)
        self.dropout2 = nn.Dropout(0.2)
        self.fc3 = nn.Linear(hidden_dim // 2, num_classes)
        
    def attention_net(self, lstm_output):
        attention_weights = torch.softmax(self.attention(lstm_output), dim=1)
        context_vector = torch.sum(attention_weights * lstm_output, dim=1)
        return context_vector
        
    def forward(self, x):
        embedded = self.embedding(x)
        embedded = self.embedding_dropout(embedded)
        lstm_out, _ = self.lstm(embedded)
        attn_out = self.attention_net(lstm_out)
        x = self.fc1(attn_out)
        x = self.bn1(x)
        x = torch.relu(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = torch.relu(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs, device):
    best_f1 = 0
    patience = 5
    patience_counter = 0
    
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        train_preds = []
        train_labels = []
        
        for batch_texts, batch_labels in train_loader:
            batch_texts, batch_labels = batch_texts.to(device), batch_labels.to(device)
            optimizer.zero_grad()
            outputs = model(batch_texts)
            loss = criterion(outputs, batch_labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            train_loss += loss.item()
            predictions = torch.argmax(outputs, dim=1)
            train_preds.extend(predictions.cpu().numpy())
            train_labels.extend(batch_labels.cpu().numpy())
        
        model.eval()
        val_loss = 0
        val_preds = []
        val_labels = []
        
        with torch.no_grad():
            for batch_texts, batch_labels in val_loader:
                batch_texts, batch_labels = batch_texts.to(device), batch_labels.to(device)
                outputs = model(batch_texts)
                loss = criterion(outputs, batch_labels)
                val_loss += loss.item()
                predictions = torch.argmax(outputs, dim=1)
                val_preds.extend(predictions.cpu().numpy())
                val_labels.extend(batch_labels.cpu().numpy())
        
        train_precision, train_recall, train_f1, _ = precision_recall_fscore_support(train_labels, train_preds, average='binary')
        train_accuracy = accuracy_score(train_labels, train_preds)
        
        val_precision, val_recall, val_f1, _ = precision_recall_fscore_support(val_labels, val_preds, average='binary')
        val_accuracy = accuracy_score(val_labels, val_preds)
        
        scheduler.step(val_loss)
        
        print(f'Epoch {epoch+1}/{num_epochs}:')
        print(f'Train Loss: {train_loss/len(train_loader):.4f}, Accuracy: {train_accuracy:.4f}, F1: {train_f1:.4f}')
        print(f'Val Loss: {val_loss/len(val_loader):.4f}, Accuracy: {val_accuracy:.4f}, F1: {val_f1:.4f}')
        print(f'Val Precision: {val_precision:.4f}, Recall: {val_recall:.4f}\n')
        
        if val_f1 > best_f1:
            best_f1 = val_f1
            torch.save({
                'model_state_dict': model.state_dict(),
                'preprocessor': preprocessor,
                'config': config
            }, 'best_model.pt')
            patience_counter = 0
        else:
            patience_counter += 1
            
        if patience_counter >= patience:
            print(f'Early stopping triggered after epoch {epoch+1}')
            break

config = {
    'max_vocab_size': 15000,
    'max_seq_length': 128,
    'embed_dim': 300,
    'hidden_dim': 300,
    'batch_size': 20,
    'learning_rate': 0.001,
    'num_epochs': 20,
    'num_lstm_layers': 3
}

train_df = pd.read_csv('train.tsv', sep='\t')
test_df = pd.read_csv('test.tsv', sep='\t')

label_mapping = {'NOCUOUS': 0, 'INNOCUOUS': 1}
train_df['Detected as'] = train_df['Detected as'].map(label_mapping)
test_df['Detected as'] = test_df['Detected as'].map(label_mapping)

preprocessor = TextPreprocessor(max_vocab_size=config['max_vocab_size'], max_seq_length=config['max_seq_length'])
preprocessor.fit(train_df['Sentence'])

X_train = preprocessor.transform(train_df['Sentence'])
X_val = preprocessor.transform(test_df['Sentence'])
y_train = torch.tensor(train_df['Detected as'].values)
y_val = torch.tensor(test_df['Detected as'].values)

train_dataset = TextDataset(X_train, y_train)
val_dataset = TextDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config['batch_size'])

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ImprovedTextClassifier(
    vocab_size=len(preprocessor.word2idx),
    embed_dim=config['embed_dim'],
    hidden_dim=config['hidden_dim'],
    num_classes=2,
    num_layers=config['num_lstm_layers']
).to(device)

class_counts = torch.bincount(y_train)
class_weights = 1.0 / class_counts
class_weights = class_weights / class_weights.sum()
class_weights = class_weights.to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.AdamW(model.parameters(), lr=config['learning_rate'], weight_decay=0.01)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)

# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler,
           config['num_epochs'], device)

ModuleNotFoundError: No module named 'torch'